<h1><center>Obtaining Stocks and Market Data + Fama-MacBeth Regression</center></h1>

## Introduction to Programming 2477-2223 S1
## Nova School of Business and Economics


### Team: Daniel Brosko (55036), ... (TODO)

TODO: theoretical description

## 1 Introduction

TODO

## 2 General packages loading

In [29]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
# from datetime import datetime
import datetime
import time
import yfinance as yf
from sklearn.linear_model import LinearRegression

#importing own functions
import scraping_functions as sf
import computing_functions as cf
import stock_prices as sp


In [34]:
# pip list # used for requirements.txt to get the version numbers

In [2]:
# loading tickers of selected stocks
tickers = sf.load_tickers("tickers_set.csv")

## 3 Market Capitalization Data Scraping

Here we scrape the market capitalizations of the firms using own scraping function from scraping_function.py.
The data are in billions of USD.

In [3]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(sf.mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Elapsed time:', round(elapsed_time,3) , 'seconds.')

# mkt_cap_df_list

Elapsed time: 5.72 seconds.


In [4]:
# merging mkt. capital for all stocks, and renaming the columns according to particular stocks
df_mkt_cap = mkt_cap_df_list[0]

for dataframe in mkt_cap_df_list[1:] :
    df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')

mc_columns_names = ['Date']
mc_columns_names.extend(tickers)
df_mkt_cap.columns = mc_columns_names
df_mkt_cap.set_index('Date', inplace=True)

df_mkt_cap

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: 

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2009-04-01,0.78,0.00,6.24,2.04,1.84,1.07,31.68,4.98,4.81,12.40,27.33,13.47,9.12,1.17,14.97,6.26,94.12,6.70,1.93,7.62
2009-04-02,0.84,0.00,6.60,2.07,1.69,1.11,32.90,5.38,4.89,12.81,28.56,13.66,8.65,1.23,15.07,6.91,92.64,7.07,2.00,8.08
2009-04-03,0.89,0.00,6.45,2.13,1.79,1.10,33.69,5.52,4.96,12.64,29.27,13.57,8.76,1.27,14.52,7.01,91.16,7.25,2.07,8.12
2009-04-06,0.88,0.00,6.32,2.16,1.66,1.09,33.61,5.47,5.00,12.75,27.93,13.38,8.79,1.21,14.32,6.92,92.24,7.05,1.93,7.96
2009-04-07,0.83,0.00,5.95,2.14,1.79,1.06,32.54,5.41,4.86,12.58,26.43,13.31,8.50,1.17,15.03,6.61,90.90,6.72,1.80,7.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,13.95,54.11,186.35,34.04,6.46,16.63,941.85,19.86,53.78,63.71,66.26,22.95,51.39,4.32,42.08,64.60,277.49,41.83,24.81,12.20
2022-11-30,14.61,54.70,189.91,35.45,6.61,17.46,983.84,20.23,54.72,64.81,67.45,23.65,51.86,4.47,44.07,65.12,281.08,42.30,25.06,12.53
2022-12-01,14.43,51.23,190.35,36.56,6.57,18.01,973.24,20.31,54.25,66.03,67.39,23.66,51.33,4.90,43.59,65.08,286.41,42.57,24.66,12.34


## 4 Getting stock prices

In [5]:
# downloading stocks prices - designed to save as csv, so we have "back-up" if something goes wrong later
success_tickers, failed_tickers = sp.get_stocks(tickers)

TER - SUCCESS
ADM - SUCCESS
TMUS - SUCCESS
IDXX - SUCCESS
LNC - SUCCESS
WST - SUCCESS
AMZN - SUCCESS
DOV - SUCCESS
ORLY - SUCCESS
BSX - SUCCESS
USB - SUCCESS
ETR - SUCCESS
AZO - SUCCESS
PVH - SUCCESS
BIIB - SUCCESS
ETN - SUCCESS
PFE - SUCCESS
WMB - SUCCESS
FITB - SUCCESS
CHRW - SUCCESS
Elapsed time: 15.302 seconds.


In [6]:
# loading stock prices into list of dataframes using own function
stocks_df_list = sp.load_stocks(success_tickers)
# stocks_df_list

In [7]:
df_close_prices = stocks_df_list[0][['Date','Adj Close']]
 
for index in range(1, len(stocks_df_list)):
    dataframe = stocks_df_list[index][['Date','Adj Close']]
    df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')

cp_columns_names = ['Date']
cp_columns_names.extend(success_tickers)
df_close_prices.columns = cp_columns_names
df_close_prices['Date'] = pd.to_datetime(df_close_prices['Date']).dt.date
df_close_prices.set_index('Date', inplace=True)

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn

In [20]:
# Making returns from daily prices
# daily_returns = df_close_prices.pct_change(1) # 1 for ONE DAY lookback
daily_returns = np.log(df_close_prices/df_close_prices.shift())
daily_returns.dropna(inplace=True)
daily_returns.head()

# monthly_return = prices.pct_change(21) # 21 for ONE MONTH lookback
# annual_return = prices.pct_change(252)

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2017-12-08,0.005197,0.004830,0.009963,0.004932,0.012170,0.000201,0.001904,0.019354,0.025045,0.028736,0.008344,0.004054,0.027455,0.012410,0.019215,0.000388,0.006738,0.002806,-0.000987,0.010922
2017-12-11,-0.001235,0.009352,0.008439,-0.005433,-0.017598,-0.005536,0.005938,-0.018631,-0.015372,-0.010533,-0.001627,0.006405,-0.018568,-0.006223,0.006913,0.006191,0.013065,0.012185,-0.002307,-0.004650
2017-12-12,0.012038,-0.003108,0.006480,-0.002696,0.005170,-0.000808,-0.003290,0.004227,-0.001538,0.004304,0.004153,-0.015729,0.003437,-0.012112,-0.000854,-0.000129,0.010166,0.006209,0.004938,0.000341
2017-12-13,-0.002934,-0.042050,0.005498,-0.006741,-0.022423,-0.002630,-0.000816,-0.010446,-0.003815,0.003897,-0.016349,0.000720,-0.004002,-0.002486,0.008991,-0.002446,0.001639,-0.000688,-0.016891,-0.002617
2017-12-14,0.004642,-0.012057,-0.020097,-0.018823,-0.001979,-0.005077,0.008664,-0.004481,-0.018344,-0.003507,-0.008276,-0.007953,-0.009905,-0.006962,-0.021888,-0.014674,-0.004925,0.021111,-0.009059,-0.022817


In [9]:
# continue work here

### 5 Market Data from prof. Kenneth French's website 

We dowloaded data about the market from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html , however, we need to change the date into suitable format.

In [21]:
data_market = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=3)
data_market = data_market[:-1]

data_market.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
data_market['Date'] = data_market['Date'].map(date_format_lambda)
data_market['Date'] = pd.to_datetime(data_market['Date']).dt.date
data_market.set_index('Date', inplace=True)
data_market.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009


In [22]:
# Filtering data for reasonable time interval
mkt_cap = df_mkt_cap.query("Date >= datetime.date(2018,1,1) and Date <= datetime.date(2022,10,31)")
returns = daily_returns.query("Date >= datetime.date(2018,1,1) and Date <= datetime.date(2022,10,31)")
market = data_market.query("Date >= datetime.date(2018,1,1) and Date <= datetime.date(2022,10,31)")

In [84]:
# adjusting stock returns to risk-free rate
risk_free_rate = market.loc[:, 'RF']
risk_free_rate = pd.DataFrame(risk_free_rate)
risk_free_rate_column = risk_free_rate.copy()

for column in range(1, len(success_tickers)):
    risk_free_rate = pd.concat([risk_free_rate, risk_free_rate_column], axis=1)

excess_returns = returns.subtract(risk_free_rate.values)

excess_returns

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.047702,-0.005751,0.002311,0.003103,0.007438,0.002377,0.010570,0.000907,0.029610,0.013573,0.002364,0.008757,0.028771,0.006745,0.041808,-0.007900,0.000056,0.015094,-0.003038,0.001269
2018-01-03,-0.001709,-0.013763,-0.012737,0.020513,-0.010632,-0.004795,0.006694,0.002031,0.012208,0.005015,0.003761,-0.017078,0.011349,-0.004274,0.010854,0.028645,0.001382,0.021243,-0.003046,-0.008790
2018-01-04,-0.001278,0.010703,-0.015636,0.005718,0.011639,-0.006301,-0.001534,-0.000261,0.009014,-0.015434,0.001669,-0.020177,0.009662,0.009751,-0.005588,-0.016713,-0.003823,0.011042,-0.002403,-0.003433
2018-01-05,0.016842,-0.012698,0.019080,0.009973,-0.003975,0.006473,0.010033,-0.013008,0.004039,0.019347,0.000165,-0.013728,0.012533,0.005739,0.001326,0.001647,-0.004099,-0.008769,-0.003392,-0.007673
2018-01-08,0.006636,-0.008243,0.001710,-0.015166,-0.007898,-0.000363,0.008322,-0.005414,-0.007076,-0.012180,-0.004555,0.005446,-0.017699,-0.008449,-0.044211,0.001833,-0.017186,-0.002310,-0.006326,0.007419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.016880,0.009619,-0.001641,0.010255,0.013569,0.040840,-0.004511,-0.002004,-0.001584,-0.003222,0.010170,0.012276,0.002356,0.047404,-0.014867,0.020155,-0.009903,0.012830,0.017114,-0.007195
2022-10-26,0.021121,0.015527,-0.009508,0.005746,-0.027826,0.009612,-0.052824,-0.008881,-0.008772,0.000795,-0.001995,-0.011286,-0.017666,-0.004292,0.013803,-0.014635,-0.000743,-0.001625,-0.006484,0.006774
2022-10-27,-0.040488,-0.005247,-0.012492,-0.017047,-0.000641,-0.150705,-0.052485,-0.007322,0.033969,-0.016038,-0.018340,-0.008044,0.028674,-0.027370,-0.020171,-0.002992,-0.017972,-0.000172,-0.015516,-0.007896


Just to remind what objects (dataframes) are important fot us now, we obtained the following dataframes that we are going to use:

* mkt_cap - market capitalization    
* excess_returns - daily stock returns adjusted to risk-free rate
* market - FF market factors

In [ ]:
for (i in 1:length(Stock_Prices))
{if(dim(stock_adj_returns_and_market[[i]])[1] > 253){
    for (k in 254:dim(stock_adj_returns_and_market[[i]][1])[1]){
        betas[[i]][k] <- as.numeric(coef(lm(stock_adj_returns_and_market[[i]][(k-253):k,3] ~ stock_adj_returns_and_market[[i]][(k-253):k,2]))[2])                                          }
        }
}

In [ ]:
#due to few technical difficulties, we use "beta_frame" list to keep the correct index and prevent unwanted changes.
beta_frame <- Stock_Returns_Daily
for (i in 1:length(Stock_Returns_Daily)) {beta_frame[[i]] <- cbind(beta_frame[[i]], beta_x[[i]])}
for (i in 1:length(Stock_Returns_Daily)) {colnames(beta_frame[[i]])[2] <- 'Beta'}
for (i in 1:length(Stock_Returns_Daily)) {beta_frame[[i]] <- beta_frame[[i]][,2]}
tail(beta_frame[[4]])

### 6 Market Betas

We will estimate stocks's sensitivity to the market (beta factors) on daily data using linear regression.

In [85]:
excess_returns_index = excess_returns.index.copy()
excess_returns.reset_index(inplace=True)
excess_returns.drop(['Date'], axis=1, inplace=True)
excess_returns

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
0,0.047702,-0.005751,0.002311,0.003103,0.007438,0.002377,0.010570,0.000907,0.029610,0.013573,0.002364,0.008757,0.028771,0.006745,0.041808,-0.007900,0.000056,0.015094,-0.003038,0.001269
1,-0.001709,-0.013763,-0.012737,0.020513,-0.010632,-0.004795,0.006694,0.002031,0.012208,0.005015,0.003761,-0.017078,0.011349,-0.004274,0.010854,0.028645,0.001382,0.021243,-0.003046,-0.008790
2,-0.001278,0.010703,-0.015636,0.005718,0.011639,-0.006301,-0.001534,-0.000261,0.009014,-0.015434,0.001669,-0.020177,0.009662,0.009751,-0.005588,-0.016713,-0.003823,0.011042,-0.002403,-0.003433
3,0.016842,-0.012698,0.019080,0.009973,-0.003975,0.006473,0.010033,-0.013008,0.004039,0.019347,0.000165,-0.013728,0.012533,0.005739,0.001326,0.001647,-0.004099,-0.008769,-0.003392,-0.007673
4,0.006636,-0.008243,0.001710,-0.015166,-0.007898,-0.000363,0.008322,-0.005414,-0.007076,-0.012180,-0.004555,0.005446,-0.017699,-0.008449,-0.044211,0.001833,-0.017186,-0.002310,-0.006326,0.007419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,0.016880,0.009619,-0.001641,0.010255,0.013569,0.040840,-0.004511,-0.002004,-0.001584,-0.003222,0.010170,0.012276,0.002356,0.047404,-0.014867,0.020155,-0.009903,0.012830,0.017114,-0.007195
1213,0.021121,0.015527,-0.009508,0.005746,-0.027826,0.009612,-0.052824,-0.008881,-0.008772,0.000795,-0.001995,-0.011286,-0.017666,-0.004292,0.013803,-0.014635,-0.000743,-0.001625,-0.006484,0.006774
1214,-0.040488,-0.005247,-0.012492,-0.017047,-0.000641,-0.150705,-0.052485,-0.007322,0.033969,-0.016038,-0.018340,-0.008044,0.028674,-0.027370,-0.020171,-0.002992,-0.017972,-0.000172,-0.015516,-0.007896
1215,0.031773,-0.003064,0.060147,0.011599,0.019633,0.008162,-0.081468,0.018704,0.016986,0.026295,0.007366,0.014382,-0.002420,0.004988,0.011842,0.013179,0.025282,-0.005783,0.006665,-0.000722


In [79]:


# df_mkt_cap.set_index('Date', inplace=True)
# excess_returns.set_index('Date', inplace=True)

KeyError: "['index'] not found in axis"

In [87]:
betas = excess_returns.copy()
# fill with NAs TODO

for stock in excess_returns.columns:
    column = 0
    for row in range(len(excess_returns[stock])-253):
        y = excess_returns.iloc[row:(row + 253), :]
        y = y.loc[:, stock].values.reshape(-1, 1)
        x = market.iloc[row:(row + 253), :]
        x = x.loc[:,'Mkt-RF'].values.reshape(-1, 1)

        model = LinearRegression(fit_intercept=False).fit(x, y)
        slope = model.coef_
        betas.iloc[row + 253,column] = slope
    column += 1
    
betas.set_index(excess_returns_index, inplace=True)
betas

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

...

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(253, 1)
(

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.047702,-0.005751,0.002311,0.003103,0.007438,0.002377,0.010570,0.000907,0.029610,0.013573,0.002364,0.008757,0.028771,0.006745,0.041808,-0.007900,0.000056,0.015094,-0.003038,0.001269
2018-01-03,-0.001709,-0.013763,-0.012737,0.020513,-0.010632,-0.004795,0.006694,0.002031,0.012208,0.005015,0.003761,-0.017078,0.011349,-0.004274,0.010854,0.028645,0.001382,0.021243,-0.003046,-0.008790
2018-01-04,-0.001278,0.010703,-0.015636,0.005718,0.011639,-0.006301,-0.001534,-0.000261,0.009014,-0.015434,0.001669,-0.020177,0.009662,0.009751,-0.005588,-0.016713,-0.003823,0.011042,-0.002403,-0.003433
2018-01-05,0.016842,-0.012698,0.019080,0.009973,-0.003975,0.006473,0.010033,-0.013008,0.004039,0.019347,0.000165,-0.013728,0.012533,0.005739,0.001326,0.001647,-0.004099,-0.008769,-0.003392,-0.007673
2018-01-08,0.006636,-0.008243,0.001710,-0.015166,-0.007898,-0.000363,0.008322,-0.005414,-0.007076,-0.012180,-0.004555,0.005446,-0.017699,-0.008449,-0.044211,0.001833,-0.017186,-0.002310,-0.006326,0.007419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.005675,0.009619,-0.001641,0.010255,0.013569,0.040840,-0.004511,-0.002004,-0.001584,-0.003222,0.010170,0.012276,0.002356,0.047404,-0.014867,0.020155,-0.009903,0.012830,0.017114,-0.007195
2022-10-26,0.005626,0.015527,-0.009508,0.005746,-0.027826,0.009612,-0.052824,-0.008881,-0.008772,0.000795,-0.001995,-0.011286,-0.017666,-0.004292,0.013803,-0.014635,-0.000743,-0.001625,-0.006484,0.006774
2022-10-27,0.005633,-0.005247,-0.012492,-0.017047,-0.000641,-0.150705,-0.052485,-0.007322,0.033969,-0.016038,-0.018340,-0.008044,0.028674,-0.027370,-0.020171,-0.002992,-0.017972,-0.000172,-0.015516,-0.007896


In [ ]:
returns

In [ ]:
market

### 6 Book to Value Ratio Scraping

In [ ]:
# TODO - download book to market ratio

In [ ]:
# TODO - download market indeces

In [ ]:
# TODO - perform regression

In [ ]:
# TODO - put results into table

In [ ]:
# TODO - comment on findings

In [ ]:
# todo - solve issues

tickers = pd.read_csv("tickers_set.csv", sep = ",")
tickers = pd.DataFrame(tickers)
tickers.columns = ["Index", "Ticker"]
# tickers.head()

tickers = tickers.Ticker
# tickers.head()

long_names = pd.DataFrame(index = range(len(tickers)), columns = 2)

for i in range(len(tickers)):
    long_names[i,0] = tickers[i]
    header = str(curlGetHeaders("https://www.macrotrends.net/stocks/charts/" + tickers[i])[5])
    long_names[i,1] = "Location: https://www.macrotrends.net/stocks/charts/" + tickers[i] + "/" + header
    long_names[i,1] = gsub("/\r\n","",long_names[i,2])
    
    
long_names

#stocks <- pd.Data.Frame()

In [ ]:
# url = https://www.macrotrends.net/stocks/charts/AAPL/apple/price-book
#page = requests.get('https://www.macrotrends.net/stocks/charts/AAPL')#/apple/price-book')


In [ ]:
get.packages('rvest')
library(rvest)

stocks <- pd.Data.Frame(read.csv("tickers_set.csv", sep=","))
colnames(stocks) <- c("Index","Ticker") #maybe pridat adresu
head(stocks,3)

stock_names <- rep(NA,length(stocks$Ticker))
for (i in seq(1:length(stocks$Ticker))){
  stock_names[i] <- as.character(stocks[i,2])
}

long_names <- data.frame(matrix(NA, nrow = length(stocks$Ticker), ncol = 2))
for (i in seq(1:length(stocks$Ticker))){
  long_names[i,1] <- stock_names[i]
  long_names[i,2] <- gsub(paste0("Location: https://www.macrotrends.net/stocks/charts/", stock_names[i],"/"),"",as.character(curlGetHeaders(paste0("https://www.macrotrends.net/stocks/charts/",stock_names[i]))[5]))
  long_names[i,2] <- gsub("/\r\n","",long_names[i,2])
}
names(long_names) <- c("Ticker", "Long_Name")
print(long_names)
write.csv(long_names,"long_names.csv")


In [ ]:
my_date = datetime.strptime('2018-01-05', '%Y-%m-%d')
my_date.strftime('%A')